In [1]:
import pandas as pd

df = pd.read_csv('frames_salah_trial.csv')

In [2]:
df.head()

,question,answer,query,result,project_id,execution_time,node_count,completion_status,eval_timestamp,server_url
0,If my future wife has the same first name as t...,Jane Ballou,If my future wife has the same first name as t...,I will not repeat any of the reasoning steps ...,5ee34f33-7656-45db-9a1a-a3faf6d8c77a,93.721762,5,completed,2025-07-11T18:30:27.053409,http://localhost:5000
1,Imagine there is a building called Bronte towe...,37th,Imagine there is a building called Bronte towe...,\n\nThe Charlotte Brontë book published in 184...,82288b5e-365a-4a90-aab9-0b7815900094,248.959409,5,completed,2025-07-11T18:30:27.053409,http://localhost:5000
2,How many years earlier would Punxsutawney Phil...,87,How many years earlier would Punxsutawney Phil...,"\n\nBased on the research conducted, Punxsutaw...",e62cd668-ed51-44c8-a954-02013e1e9898,180.489119,4,completed,2025-07-11T18:30:27.053409,http://localhost:5000
3,"As of August 1, 2024, which country were holde...",France,"As of August 1, 2024, which country were holde...","\nAs of August 1, 2024, the country that held ...",58ea534f-96ad-45af-93cf-bdfc1367e266,119.789040,4,completed,2025-07-11T18:30:27.053409,http://localhost:5000
4,What is the name of the vocalist from the firs...,Jens Kidman,What is the name of the vocalist from the firs...,I will use the information from my reasoning ...,efeb922d-8b8a-4b7d-9c28-ac6af5b2557f,115.867239,4,completed,2025-07-11T18:30:27.053409,http://localhost:5000


In [7]:
sample = df.sample(1)
question = sample['question'].values[0]
answer = sample['result'].values[0]

In [4]:
from litellm import completion
def extract_final_answer(query: str, result: str, model: str= "openrouter/google/gemini-2.5-flash") -> str:
    """Extract the final answer from a verbose result using LiteLLM."""
        
    prompt = f"""Given the following query and verbose result, extract ONLY the final answer. 
    Return just the exact final answer with no additional text, explanations, or formatting.

    Query: {query}

    Verbose Result: {result}

    Final Answer:"""

    response = completion(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4,
    )

    final_answer = response.choices[0].message.content.strip()
    return final_answer

In [8]:
final_answer = extract_final_answer(question, answer)

/Users/salahalzubi/cursor_projects/SentientResearchAgent/.venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='95.35%',...one, 'reasoning': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...finish_reason': 'STOP'}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


In [13]:
extracted_df = pd.read_csv("frames_salah_trial_extracted.csv")

In [15]:
extracted_df[['question', 'answer', 'extracted_answer']]

,question,answer,extracted_answer
0,If my future wife has the same first name as t...,Jane Ballou,Jane Ballou
1,Imagine there is a building called Bronte towe...,37th,35th-tallest building in New York City
2,How many years earlier would Punxsutawney Phil...,87,87
3,"As of August 1, 2024, which country were holde...",France,France
4,What is the name of the vocalist from the firs...,Jens Kidman,Mitch Lucker
...,...,...,...
145,"As of August 3, 2024, what is the sum of the b...",5980,1909
146,Rosie Ruiz was disqualified from the Boston Ma...,Jacqueline Gareau placed first in the 1988 Gra...,1st
147,"On Nov 15, 2017 a painting was sold for US $45...",1766,1766
148,A disease that had millions of dollars raised ...,1981,1981


In [16]:
graded_df = pd.read_csv("frames_salah_trial_extracted_graded.csv")

In [19]:
graded_df.grade.value_counts(normalize=True)

grade
1    0.586667
0    0.413333
Name: proportion, dtype: float64

In [20]:
graded_df[graded_df['grade'] == 0]

,question,answer,query,result,project_id,execution_time,node_count,completion_status,eval_timestamp,server_url,extracted_answer,grade
1,Imagine there is a building called Bronte towe...,37th,Imagine there is a building called Bronte towe...,\n\nThe Charlotte Brontë book published in 184...,82288b5e-365a-4a90-aab9-0b7815900094,248.959409,5,completed,2025-07-11T18:30:27.053409,http://localhost:5000,35th-tallest building in New York City,0
4,What is the name of the vocalist from the firs...,Jens Kidman,What is the name of the vocalist from the firs...,I will use the information from my reasoning ...,efeb922d-8b8a-4b7d-9c28-ac6af5b2557f,115.867239,4,completed,2025-07-11T18:30:27.053409,http://localhost:5000,Mitch Lucker,0
6,I have an element in mind and would like you t...,Mendelevium is named after Dmitri Mendeleev.,I have an element in mind and would like you t...,\nThe element with atomic number 106 is **seab...,3d40222d-aebd-4c91-a5af-d85a399fcc1b,138.037470,5,completed,2025-07-11T18:30:27.053409,http://localhost:5000,Glenn T. Seaborg,0
8,A general motors vehicle is named after the la...,4,A general motors vehicle is named after the la...,No output generated,7bf18bf3-7d9a-405b-992b-3cf5731f9a5f,198.480795,6,completed,2025-07-11T18:30:27.053409,http://localhost:5000,12,0
15,In Slovakia there is a well known Film Festiva...,Roudnice nad Labem,In Slovakia there is a well known Film Festiva...,Planned 3 sub-tasks: Identify the film that wo...,6a5b6708-35b2-409e-8b33-a6d5a8b9addc,43.298700,4,running,2025-07-11T18:30:27.053409,http://localhost:5000,Paris,0
...,...,...,...,...,...,...,...,...,...,...,...,...
141,"Suppose Egon Sendler's book ""Les mystères du C...",51 years.,"Suppose Egon Sendler's book ""Les mystères du C...",I will use the direct answer format.\nBased o...,3d70fcb7-c39f-4169-a4c1-125d7ea72c7a,178.883230,4,completed,2025-07-11T18:30:27.053409,http://localhost:5000,-28 years,0
142,"As of August 5, 2024, what is the name of the ...",The Sherman Antitrust Act,"As of August 5, 2024, what is the name of the ...","\nAs of August 5, 2024, the federal law that B...",e019e7b1-7ab0-47f4-8253-7095f1948a70,104.134771,4,completed,2025-07-11T18:30:27.053409,http://localhost:5000,National Labor Relations Act (NLRA),0
145,"As of August 3, 2024, what is the sum of the b...",5980,"As of August 3, 2024, what is the sum of the b...","\nBased on the provided sub-task results, the ...",bdd48db0-aea6-4b24-9753-3da3bbbe80b5,138.472103,4,completed,2025-07-11T18:30:27.053409,http://localhost:5000,1909,0
146,Rosie Ruiz was disqualified from the Boston Ma...,Jacqueline Gareau placed first in the 1988 Gra...,Rosie Ruiz was disqualified from the Boston Ma...,\nAfter Rosie Ruiz's disqualification from the...,e4934e95-8904-42d8-a5df-1956ea747c43,128.335218,4,completed,2025-07-11T18:30:27.053409,http://localhost:5000,1st,0
